In [ ]:
import os
from pathlib import Path

try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass

base_path = os.getenv("LSM_BASE")
if not base_path:
    raise ValueError("❌ Environment variable 'LSM_BASE' is not set!")

model_path         = os.path.join(base_path, "models", "hand_landmarker.task")
raw_data_path      = os.path.join(base_path, "data", "raw")
# static_json_path   = os.path.join(base_path, "data", "metadata", "static_gestures.json")
# dynamic_json_path  = os.path.join(base_path, "data", "metadata", "dynamic_gestures.json")

# 3) Gather all image files under data/raw/<letter>/
# image_records list is in lexicographical order by folder name, then lexicographical order by file name
valid_exts = {".jpg", ".jpeg", ".png"}
image_records = []  # list of (letter, path)

for letter_dir in sorted(os.listdir(raw_data_path)):
    letter_path = os.path.join(raw_data_path, letter_dir)
    if not os.path.isdir(letter_path):
        continue
    for file in sorted(os.listdir(letter_path)):
        ext = Path(file).suffix.lower()
        if ext in valid_exts:
            image_records.append((letter_dir, os.path.join(letter_path, file)))


In [ ]:
# 4) Quick check
print(f"Found {len(image_records)} image files in {raw_data_path!r}")
print("First 10 records:")
for rec in image_records[:10]:
    print(" ", rec)

In [ ]:
# Cell 2: 🧪 Define preprocessing functions and test on one image

import cv2
import numpy as np
import matplotlib.pyplot as plt


# Preprocessing functions
def adjust_lighting(image):
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    ycrcb[:, :, 0] = cv2.equalizeHist(ycrcb[:, :, 0])
    return cv2.cvtColor(ycrcb, cv2.COLOR_YCrCb2BGR)

def gamma_correction(image, gamma=1.5):
    inv_gamma = 1.0 / gamma
    table = np.array([((i/255.0)**inv_gamma)*255 for i in np.arange(256)]).astype("uint8")
    return cv2.LUT(image, table)

def attenuate_red(image):
    b, g, r = cv2.split(image)
    r = (r * 0.3).astype(np.uint8)
    return cv2.merge([b, g, r])

def resize_and_pad(image, size=(640, 480)):
    h, w = image.shape[:2]
    scale = min(size[0]/w, size[1]/h)
    new_w, new_h = int(w*scale), int(h*scale)
    resized = cv2.resize(image, (new_w, new_h))
    top = (size[1] - new_h) // 2
    bottom = size[1] - new_h - top
    left = (size[0] - new_w) // 2
    right = size[0] - new_w - left
    return cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0,0,0))

def full_preprocess(image):
    img = adjust_lighting(image)
    img = gamma_correction(img)
    img = attenuate_red(img)
    img = resize_and_pad(img)
    return img

# Load a sample image
# Replace this with the first path from your image_records
sample_letter, sample_path = image_records[0]
orig = cv2.imread(sample_path)

# Apply each step
step1 = adjust_lighting(orig)
step2 = gamma_correction(step1)
step3 = attenuate_red(step2)
step4 = resize_and_pad(step3)
step_full = full_preprocess(orig)

# Prepare for display (convert BGR -> RGB)
images = [orig, step1, step2, step3, step4, step_full]
titles = ['Original','Lighting','Gamma','Red Attenuated','Resized','Full Preprocess']
images_rgb = [cv2.cvtColor(im, cv2.COLOR_BGR2RGB) for im in images]

# Plot results
fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.ravel()
for ax, title, im in zip(axes, titles, images_rgb):
    ax.imshow(im)
    ax.set_title(title)
    ax.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
# Initialize MediaPipe Hands
import mediapipe as mp

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.35
)

print("✅ MediaPipe Hands initialized:", hands)


In [ ]:
import os
import cv2
import json
from pathlib import Path
from datetime import datetime, timezone
from pandas import DataFrame

# LSM: letters that REQUIRE motion
DYNAMIC_LETTERS = {"J", "K", "M", "Q", "X", "Z"}

# 1) Build one unified list
all_entries = []

for letter, img_path in image_records[:5]:   # or [:] for all images
    img  = cv2.imread(img_path)
    proc = resize_and_pad(img)                # from Cell 2

    # Run detection
    rgb    = cv2.cvtColor(proc, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    # Prepare handedness, landmarks, confidences
    landmarks_dict  = {"right_hand": None, "left_hand": None}
    confidence_dict = {"right_hand": None, "left_hand": None}
    handedness_list = []

    if result.multi_hand_landmarks and result.multi_handedness:
        for lm_list, hand_h in zip(result.multi_hand_landmarks, result.multi_handedness):
            side = hand_h.classification[0].label.lower()  # "right" or "left"
            handedness_list.append(side)
            landmarks_dict[f"{side}_hand"] = [
                {"x": lm.x, "y": lm.y, "z": lm.z}
                for lm in lm_list.landmark
            ]
            confidence_dict[f"{side}_hand"] = hand_h.classification[0].score

    # Compose the entry
    entry = {
        "image":                Path(img_path).name,
        "letter":               letter,
        "gesture_type":         "dynamic" if letter in DYNAMIC_LETTERS else "static",
        "timestamp":            datetime.now(timezone.utc).isoformat(),
        "image_size":           list(proc.shape[1::-1]),   # [width, height]
        "hand_count":           len(handedness_list),
        "handedness_detected":  handedness_list,
        "hand_confidence":      confidence_dict,
        "landmarks":            landmarks_dict
    }
    all_entries.append(entry)

# 2) Preview in a DataFrame
DataFrame(all_entries)

# 3) Save to a single JSON file
output_path = os.path.join(base_path, "data", "metadata", "gestures.json")
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(all_entries, f, indent=4)

print(f"✅ Saved {len(all_entries)} entries to:\n  {output_path}")
